In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import helpers as h
import re
from collections import Counter
import math

## Initial exploration

In [4]:
data = h.read_input()

In [5]:
data.head(n=12)

,information
0,Tile 3923:
1,...##.....
2,.##.#..#.#
3,###....#..
4,...#...#.#
5,....#.#...
6,..........
7,.....#.###
8,.##.#..#..
9,..##..#...


Each tile is given by 11 rows: 1 with the tile name and 10 with the elements

In [6]:
len(data) / 11

144.0

In total,a 12x12 image (144 titles) is saved in a file with 11*144=1584 rows

In [7]:
data["information"].iloc[0]

'Tile 3923:'

We will enumerate the tiles from 1 to 144 as they appear in the file

In [8]:
x = h.get_image(1, data)

In [170]:
x

1     ...##.....
2     .##.#..#.#
3     ###....#..
4     ...#...#.#
5     ....#.#...
6     ..........
7     .....#.###
8     .##.#..#..
9     ..##..#...
10    ..#.#.##..
Name: information, dtype: object

Each image is tranformed into a matrix of 1 and 0 to operate faster: '.' = 1 and '#' = 0

In [9]:
m=h.image_to_matrix(x)
m

array([[1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 0, 0, 1, 1]])

In [10]:
x2 = h.matrix_to_image(m)

In [173]:
x2

0    ...##.....
1    .##.#..#.#
2    ###....#..
3    ...#...#.#
4    ....#.#...
5    ..........
6    .....#.###
7    .##.#..#..
8    ..##..#...
9    ..#.#.##..
Name: information, dtype: object

In [11]:
h.get_tile_number(1, data)

3923

In [12]:
# number of tiles 12 x 12 image
len(data) // 11

144

In [13]:
all_tiles = [(k, h.get_tile_number(k, data)) for k in range(1, 145)]

Each tile, can be rotated and fliped to obtain a total of different 8 matrices (dihedral group D₄: 4 rotations and 4 flips)

In [14]:
all_m = h.transform(m)

In [15]:
all_m

Original           [[1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [1, 0, 0, 1, ...
90° Rotation       [[1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 0, 1, ...
180° Rotation      [[1, 1, 0, 0, 1, 0, 1, 0, 1, 1], [1, 1, 1, 0, ...
270° Rotation      [[1, 0, 1, 0, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, ...
Horizontal Flip    [[1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [0, 1, 0, 1, ...
Vertical Flip      [[1, 1, 0, 1, 0, 1, 0, 0, 1, 1], [1, 1, 0, 0, ...
d1                 [[1, 1, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, ...
d2                 [[1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 1, ...
dtype: object

The boundaries can be transformed into decimal values with high bit the one the most on the left or the most on the right

In [16]:
h.boundary_to_decimal(m)

[927, 695, 851, 895]

The boundary at the right (2) reads

In [17]:
m[:,-1]

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1])

In [18]:
binary_boundary="".join(map(str, m[:,-1]))

In [19]:
int(binary_boundary, 2)

695

For a given piece, we have a total of 8 different pieces that can be generated by rotation and flipping

In [20]:
[h.boundary_to_decimal(m2) for m2 in all_m]

[[927, 695, 851, 895],
 [1019, 927, 949, 851],
 [811, 1019, 999, 949],
 [695, 811, 895, 999],
 [999, 895, 811, 695],
 [851, 949, 927, 1019],
 [949, 999, 1019, 811],
 [895, 851, 695, 927]]

In [21]:
m

array([[1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 0, 0, 1, 1]])

In [22]:
''.join(map(str, m[0]))

'1110011111'

Before working with the full example (12x12), we work the 3x3 example

## Example

In [23]:
data_example = h.read_input('input_example.txt')

In [24]:
data_example['information']

0     Tile 2311:
1     ..##.#..#.
2     ##..#.....
3     #...##..#.
4     ####.#...#
         ...    
94    .#...#.##.
95    #.#####.##
96    ..#.###...
97    ..#.......
98    ..#.###...
Name: information, Length: 99, dtype: object

In [25]:
tile_numbers = [tile[5:9] for i, tile in enumerate(data_example['information']) if i % 11 ==0]

In [26]:
print(tile_numbers)

['2311', '1951', '1171', '1427', '1489', '2473', '2971', '2729', '3079']


In [27]:
[(k, h.get_tile_number(k, data_example)) for k in range(1, 10)]

[(1, 2311),
 (2, 1951),
 (3, 1171),
 (4, 1427),
 (5, 1489),
 (6, 2473),
 (7, 2971),
 (8, 2729),
 (9, 3079)]

In total, we have 9 tiles

### Boundaries

Each piece, taking into account the 6 possible transformations (4 rotations and 2 flips), can generate up to 8 different shapes, that result in 8 different boundaries also

In [28]:
n_tiles = 9
z = []
for i in range(1, n_tiles + 1):
    tile = h.get_tile_number(i, data_example)
    img = h.get_image(i, data_example)
    m = h.image_to_matrix(img)
    m_all = h.transform(m)
    for j in range(8):
        boundary = h.boundary_to_decimal(m_all.iloc[j])
        for k in range(len(boundary)):
            z.append([tile, j+1, k+1, boundary[k]])
all_boundaries = pd.DataFrame(z, columns=['id', 'transformation', 'boundary', 'value'])

In [29]:
all_boundaries = h.get_all_boundaries(data_example)

In [30]:
all_boundaries.head()

,id,transformation,boundary,value
0,2311,1,1,813
1,2311,1,2,934
2,2311,1,3,792
3,2311,1,4,525
4,2311,2,1,705


In [31]:
np.unique(all_boundaries['value'])

array([  57,   61,   75,   99,  121,  175,  182,  313,  321,  334,  343,
        407,  436,  447,  458,  459,  481,  491,  522,  525,  542,  567,
        624,  626,  632,  675,  705,  723,  735,  752,  759,  789,  792,
        813,  839,  840,  846,  862,  907,  927,  934,  938,  945,  957,
        980,  999, 1005, 1014])

In [32]:
all_boundaries_data = h.get_all_boundaries(data)
b_unique = np.unique(all_boundaries_data['value'])

In [33]:
unique_boundaries = [int(x) for x in b_unique] 

We have to prove that all boundary values are not symatrical and that all boundary values and its reversed values are different

In [34]:
# Generate all 10-digit binary palindromes and convert them to decimal

def generate_binary_palindromes(n_bits):
    half = n_bits // 2
    palindromes = []
    for i in range(2**half):
        left = f"{i:0{half}b}"
        full = left + left[::-1]
        palindromes.append(int(full, 2))
    return palindromes

# Generate for 10-bit binary palindromes
palindromes_10_bit = generate_binary_palindromes(10)
palindromes_10_bit

[0,
 48,
 72,
 120,
 132,
 180,
 204,
 252,
 258,
 306,
 330,
 378,
 390,
 438,
 462,
 510,
 513,
 561,
 585,
 633,
 645,
 693,
 717,
 765,
 771,
 819,
 843,
 891,
 903,
 951,
 975,
 1023]

In [35]:
for x in palindromes_10_bit:
    if x in unique_boundaries:
        print(x)

All boundary values are non symmetric, that is that the boundary values and its reverse are not identical. therefore, no boundary in the exemple is symmetric.

### Pieces

Each piece can put in 8 different shapes (after combinations of 4 rotations and 2 flips)

In [36]:
all_transforms = h.get_all_transforms(data_example)

In [37]:
all_transforms

,id,transformation,boundaries
0,2311,1,"[813, 934, 792, 525]"
1,2311,2,"[705, 813, 407, 792]"
2,2311,3,"[99, 705, 723, 407]"
3,2311,4,"[934, 99, 525, 723]"
4,2311,5,"[723, 525, 99, 934]"
...,...,...,...
67,3079,4,"[759, 907, 407, 522]"
68,3079,5,"[522, 407, 907, 759]"
69,3079,6,"[839, 957, 321, 934]"
70,3079,7,"[957, 522, 934, 907]"


In [38]:
n=3
[len(Counter(all_boundaries['value'][(n*32):((n+1)*32)])) for n in range(0,9)]

[8, 8, 8, 8, 8, 8, 8, 8, 8]

In [39]:
n=0
Counter(all_boundaries['value'][range(0+n*32,(n+1)*32)])

Counter({813: 4, 934: 4, 792: 4, 525: 4, 705: 4, 407: 4, 99: 4, 723: 4})

### Coincidences

In [40]:
all_transforms[all_transforms['id'] == 2311]

,id,transformation,boundaries
0,2311,1,"[813, 934, 792, 525]"
1,2311,2,"[705, 813, 407, 792]"
2,2311,3,"[99, 705, 723, 407]"
3,2311,4,"[934, 99, 525, 723]"
4,2311,5,"[723, 525, 99, 934]"
5,2311,6,"[792, 407, 813, 705]"
6,2311,7,"[407, 723, 705, 99]"
7,2311,8,"[525, 792, 934, 813]"


A coincidence is when two pieces share a common boundary. This function computes the number of other pieces that have the same boundary b in piece id. For instance, boundary 813 in piece 2311 is found in another piece.

In [41]:
h.n_coincidences(813, 2311, all_boundaries)

1

In [42]:
id = 2311 # tile
b = 813 # boundary value
new_df = all_boundaries[all_boundaries['id']!=id] 
new_df[new_df['value'] == b][['id','transformation', 'boundary']]


,id,transformation,boundary
98,1427,1,3
103,1427,2,4
116,1427,6,1
125,1427,8,2


Boundary 813 exists also in tile 1427 in 4 different boundary positions. Given a boundary value, and a boundary position there is only one tile shape possible. This is the key to join all pieces to create the final matrix. For all possible boundary values, we find how many other pieces have the same boundary. 

In [43]:
# all coincidence for all boundaries for each piece and boundary
all_coincidences = [h.n_coincidences(int(r[3]), int(r[0]), all_boundaries) for r in all_boundaries.values]
np.unique(all_coincidences).tolist()

[0, 1]

Each piece has a maximum of one common border with any other piece

### Number of common boundaries in a tile 

We compute it only for the tiles in the original shape. To match them, we will have to transform them

In [44]:
y = all_transforms
i_count = 0
z=[]
for tile in y.values :
    if i_count % 8 == 0:
        n_matches = 0
        for b in tile[2]:
            n_matches = n_matches + h.n_coincidences(b, tile[0], all_boundaries)
        print([tile[0], tile[1], n_matches])
        z.append([tile[0], tile[1], n_matches])
    i_count = i_count + 1

[2311, 1, 3]
[1951, 1, 2]
[1171, 1, 2]
[1427, 1, 4]
[1489, 1, 3]
[2473, 1, 3]
[2971, 1, 2]
[2729, 1, 3]
[3079, 1, 2]


In [45]:
print(z)

[[2311, 1, 3], [1951, 1, 2], [1171, 1, 2], [1427, 1, 4], [1489, 1, 3], [2473, 1, 3], [2971, 1, 2], [2729, 1, 3], [3079, 1, 2]]


The pieces with 2 common boundaries are corners. The ones with 3 are the pieces in the boundary and the ones with 4 are the pieces inside. In this case, the corners are the pieces 1951, 1171, 2971 and 3079

In [46]:
corners = []
for x in z:
    if x[2] == 2:
        corners.append(x[0])
corners


[1951, 1171, 2971, 3079]

In [47]:
math.prod(corners)

20899048083289

This is the solution of question 1


In [48]:
data = h.read_input()
all_boundaries_data = h.get_all_boundaries(data)
all_transforms_data = h.get_all_transforms(data)

y_data = all_transforms_data
i_count = 0
z_data=[]
for tile in y_data.values :
    if i_count % 8 == 0:
        n_matches = 0
        for b in tile[2]:
            n_matches = n_matches + h.n_coincidences(b, tile[0], all_boundaries_data)
        # print([tile[0], tile[1], n_matches])
        z_data.append([tile[0], tile[1], n_matches])
    i_count = i_count + 1

In [49]:
corners_data = []
for x in z_data:
    if x[2] == 2:
        corners_data.append(x[0])
corners_data

[2243, 1213, 2953, 2273]

In [50]:
math.prod(corners_data)

18262194216271

## Second question.

We have first to build the complete image with the right pieces with the right transform in each position.

We solve it for the simple example first (3x3). Let us take the four corner tiles and identify whihc boundaries are external

In [51]:
all_boundaries[all_boundaries['id'].isin(corners)]

,id,transformation,boundary,value
32,1951,1,1,313
33,1951,1,2,525
34,1951,1,3,459
35,1951,1,4,182
36,1951,2,1,436
...,...,...,...,...
283,3079,7,4,907
284,3079,8,1,407
285,3079,8,2,839
286,3079,8,3,759


In [52]:
all_transforms[all_transforms['id'].isin(corners)]

,id,transformation,boundaries
8,1951,1,"[313, 525, 459, 182]"
9,1951,2,"[436, 313, 705, 459]"
10,1951,3,"[846, 436, 626, 705]"
11,1951,4,"[525, 846, 182, 626]"
12,1951,5,"[626, 182, 846, 525]"
13,1951,6,"[459, 705, 313, 436]"
14,1951,7,"[705, 626, 436, 846]"
15,1951,8,"[182, 459, 525, 313]"
16,1171,1,"[57, 735, 999, 121]"
17,1171,2,"[632, 57, 1005, 999]"


In [53]:
# all coincidence for all boundaries for each piece and boundary
all_boundaries_corners = all_boundaries[all_boundaries['id'].isin(corners)]
all_coincidences = [(int(r[0]),int(r[1]),int(r[2]), int(r[3]), h.n_coincidences(int(r[3]), int(r[0]), all_boundaries)) for r in all_boundaries_corners.values]
#np.unique(all_coincidences).tolist()
boundary_corner = [x for x in all_coincidences if x[4]==0]

In [54]:
boundary_corner

[(1951, 1, 3, 459, 0),
 (1951, 1, 4, 182, 0),
 (1951, 2, 1, 436, 0),
 (1951, 2, 4, 459, 0),
 (1951, 3, 1, 846, 0),
 (1951, 3, 2, 436, 0),
 (1951, 4, 2, 846, 0),
 (1951, 4, 3, 182, 0),
 (1951, 5, 2, 182, 0),
 (1951, 5, 3, 846, 0),
 (1951, 6, 1, 459, 0),
 (1951, 6, 4, 436, 0),
 (1951, 7, 3, 436, 0),
 (1951, 7, 4, 846, 0),
 (1951, 8, 1, 182, 0),
 (1951, 8, 2, 459, 0),
 (1171, 1, 3, 999, 0),
 (1171, 1, 4, 121, 0),
 (1171, 2, 1, 632, 0),
 (1171, 2, 4, 999, 0),
 (1171, 3, 1, 927, 0),
 (1171, 3, 2, 632, 0),
 (1171, 4, 2, 927, 0),
 (1171, 4, 3, 121, 0),
 (1171, 5, 2, 121, 0),
 (1171, 5, 3, 927, 0),
 (1171, 6, 1, 999, 0),
 (1171, 6, 4, 632, 0),
 (1171, 7, 3, 632, 0),
 (1171, 7, 4, 927, 0),
 (1171, 8, 1, 121, 0),
 (1171, 8, 2, 999, 0),
 (2971, 1, 1, 862, 0),
 (2971, 1, 4, 567, 0),
 (2971, 2, 1, 945, 0),
 (2971, 2, 2, 862, 0),
 (2971, 3, 2, 945, 0),
 (2971, 3, 3, 491, 0),
 (2971, 4, 3, 567, 0),
 (2971, 4, 4, 491, 0),
 (2971, 5, 1, 491, 0),
 (2971, 5, 2, 567, 0),
 (2971, 6, 3, 862, 0),
 (2971, 6, 

In [55]:
canvas=[]
direction = 'down'
searched_boundary = 1 
# First corner
# position (1, 1)
x = [1951, 2]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
n1_piece = n_piece
canvas.append(x)

In [56]:
n_piece['boundaries']

9    [436, 313, 705, 459]
Name: boundaries, dtype: object

In [57]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundary + 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundary)]
next_tile.iloc[0]

id                2311
transformation       2
boundary             1
value              705
Name: 4, dtype: int64

In [58]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
n1_piece = n_piece
canvas.append(x)

In [59]:
x

[2311, 2]

In [60]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundary + 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundary)]
next_tile.iloc[0]

id                3079
transformation       8
boundary             1
value              407
Name: 284, dtype: int64

In [61]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
n1_piece = n_piece
canvas.append(x)

In [62]:
x

[3079, 8]

In [63]:
canvas

[[1951, 2], [2311, 2], [3079, 8]]

In [64]:
direction = 'right'
searched_boundaries = [2, 4] 
# First corner
# position (1, 1)
x = canvas[0]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]


In [65]:
n_piece

,id,transformation,boundaries
9,1951,2,"[436, 313, 705, 459]"


In [66]:
n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]

313

In [67]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]

id                2729
transformation       2
boundary             4
value              313
Name: 231, dtype: int64

In [68]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)
direction = 'down'
searched_boundaries = [3, 1] 

In [69]:
canvas

[[1951, 2], [2311, 2], [3079, 8], [2729, 2]]

In [70]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]

id                1427
transformation       2
boundary             1
value             1014
Name: 100, dtype: int64

In [71]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [72]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [73]:
direction = 'right'
searched_boundaries = [2, 4]
x = canvas[3]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [74]:
canvas

[[1951, 2], [2311, 2], [3079, 8], [2729, 2], [1427, 2], [2473, 3], [2971, 2]]

In [75]:
direction = 'down'
searched_boundaries = [3, 1]
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [76]:
direction = 'down'
searched_boundaries = [3, 1]
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [77]:
canvas

[[1951, 2],
 [2311, 2],
 [3079, 8],
 [2729, 2],
 [1427, 2],
 [2473, 3],
 [2971, 2],
 [1489, 2],
 [1171, 4]]

In [95]:
all_boundaries = h.get_all_boundaries(data_example)
all_transforms = h.get_all_transforms(data_example)

n = 3
canvas=[]
#direction = 'down'
#searched_boundary = 1 
# First corner
# Choose one that can fit in top-left corner
x_initial = [1951, 2]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
for col in range(n):
    for row in range(n):
        if (col == 0) & (row == 0):
            x = x_initial
            n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
            canvas.append(x)
        else:
            next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) &
                                        (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
            x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
            n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
            canvas.append(x)
        direction = 'down'
        searched_boundaries = [3, 1]
    direction = 'right'
    searched_boundaries = [2, 4] 
    x = canvas[col * n]
    n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]  

In [96]:
canvas

[[1951, 2],
 [2311, 2],
 [3079, 8],
 [2729, 2],
 [1427, 2],
 [2473, 3],
 [2971, 2],
 [1489, 2],
 [1171, 4]]

In [97]:
canvas_example = canvas

### Canvas for data

In [89]:
data = h.read_input()
all_boundaries_data = h.get_all_boundaries(data)
all_transforms_data = h.get_all_transforms(data)

In [90]:
y_data = all_transforms_data
i_count = 0
z_data=[]
for tile in y_data.values :
    if i_count % 8 == 0:
        n_matches = 0
        for b in tile[2]:
            n_matches = n_matches + h.n_coincidences(b, tile[0], all_boundaries_data)
        # print([tile[0], tile[1], n_matches])
        z_data.append([tile[0], tile[1], n_matches])
    i_count = i_count + 1

In [91]:
corners_data = []
for x in z_data:
    if x[2] == 2:
        corners_data.append(x[0])
corners_data

[2243, 1213, 2953, 2273]

In [92]:
# all coincidence for all boundaries for each piece and boundary
all_boundaries_corners_data = all_boundaries_data[all_boundaries_data['id'].isin(corners_data)]
all_coincidences_data = [(int(r[0]),int(r[1]),int(r[2]), int(r[3]), h.n_coincidences(int(r[3]), int(r[0]), all_boundaries_data)) for r in all_boundaries_corners_data.values]
#np.unique(all_coincidences).tolist()
boundary_corner_data = [x for x in all_coincidences_data if x[4]==0]

In [93]:
boundary_corner_data

[(2243, 1, 1, 844, 0),
 (2243, 1, 2, 175, 0),
 (2243, 2, 2, 844, 0),
 (2243, 2, 3, 980, 0),
 (2243, 3, 3, 203, 0),
 (2243, 3, 4, 980, 0),
 (2243, 4, 1, 175, 0),
 (2243, 4, 4, 203, 0),
 (2243, 5, 1, 203, 0),
 (2243, 5, 4, 175, 0),
 (2243, 6, 2, 980, 0),
 (2243, 6, 3, 844, 0),
 (2243, 7, 1, 980, 0),
 (2243, 7, 2, 203, 0),
 (2243, 8, 3, 175, 0),
 (2243, 8, 4, 844, 0),
 (1213, 1, 3, 53, 0),
 (1213, 1, 4, 344, 0),
 (1213, 2, 1, 106, 0),
 (1213, 2, 4, 53, 0),
 (1213, 3, 1, 688, 0),
 (1213, 3, 2, 106, 0),
 (1213, 4, 2, 688, 0),
 (1213, 4, 3, 344, 0),
 (1213, 5, 2, 344, 0),
 (1213, 5, 3, 688, 0),
 (1213, 6, 1, 53, 0),
 (1213, 6, 4, 106, 0),
 (1213, 7, 3, 106, 0),
 (1213, 7, 4, 688, 0),
 (1213, 8, 1, 344, 0),
 (1213, 8, 2, 53, 0),
 (2953, 1, 1, 369, 0),
 (2953, 1, 2, 789, 0),
 (2953, 2, 2, 369, 0),
 (2953, 2, 3, 675, 0),
 (2953, 3, 3, 570, 0),
 (2953, 3, 4, 675, 0),
 (2953, 4, 1, 789, 0),
 (2953, 4, 4, 570, 0),
 (2953, 5, 1, 570, 0),
 (2953, 5, 4, 789, 0),
 (2953, 6, 2, 675, 0),
 (2953, 6, 3, 3

In [86]:
all_transforms = all_transforms_data
all_boundaries = all_boundaries_data

n = 12
canvas=[]
#direction = 'down'
#searched_boundary = 1 
# First corner
# Choose one that can fit in top-left corner
x_initial = [2243, 4]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
for col in range(n):
    for row in range(n):
        if (col == 0) & (row == 0):
            x = x_initial
            n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
            canvas.append(x)
        else:
            next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) &
                                        (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
            x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
            n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
            canvas.append(x)
        direction = 'down'
        searched_boundaries = [3, 1]
    direction = 'right'
    searched_boundaries = [2, 4] 
    x = canvas[col * n]
    n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]  

In [87]:
canvas

[[2243, 4],
 [2707, 3],
 [1481, 6],
 [3559, 3],
 [1489, 6],
 [1583, 5],
 [1307, 1],
 [2549, 6],
 [2677, 4],
 [1289, 1],
 [3511, 1],
 [1213, 7],
 [2663, 4],
 [1549, 6],
 [3821, 4],
 [2161, 7],
 [2003, 6],
 [1471, 6],
 [3769, 1],
 [2207, 3],
 [1933, 5],
 [3329, 1],
 [3187, 5],
 [3259, 3],
 [2087, 1],
 [1409, 4],
 [1399, 7],
 [3833, 5],
 [1873, 2],
 [1601, 4],
 [1801, 3],
 [1019, 2],
 [3853, 3],
 [2237, 1],
 [2099, 7],
 [1531, 7],
 [3067, 6],
 [3343, 6],
 [2477, 1],
 [2609, 3],
 [1831, 7],
 [1063, 4],
 [1097, 3],
 [3631, 3],
 [2551, 3],
 [2591, 2],
 [1787, 6],
 [1867, 4],
 [3881, 4],
 [3491, 5],
 [2753, 7],
 [3557, 5],
 [1657, 2],
 [3923, 4],
 [3643, 6],
 [1597, 2],
 [1259, 6],
 [1931, 5],
 [1979, 2],
 [2441, 2],
 [2333, 6],
 [3467, 4],
 [2063, 5],
 [2843, 5],
 [1447, 7],
 [1879, 2],
 [3209, 5],
 [3767, 1],
 [3541, 6],
 [2357, 4],
 [2711, 6],
 [1621, 6],
 [1607, 7],
 [2411, 6],
 [1009, 3],
 [3037, 6],
 [1733, 1],
 [3313, 5],
 [1181, 3],
 [3251, 2],
 [2749, 4],
 [3929, 2],
 [3023, 2],
 [16

In [88]:
canvas_data = canvas

### Generate complete canvas

In [287]:
# For the example data
canvas = canvas_example
data_all = data_example 
# For the real data
canvas = canvas_data
data_all = data 
# tiles in canvas
n_elements = len(canvas)
# nxn size of the canvas 
n = int(math.sqrt(n_elements))
# all tiles
all_tiles = [(k+1, h.get_tile_number(k+1, data_all)) for k in range(n_elements)]

In [279]:
all_tiles

[(1, 3923),
 (2, 1229),
 (3, 1801),
 (4, 3643),
 (5, 2141),
 (6, 1289),
 (7, 3373),
 (8, 1399),
 (9, 3259),
 (10, 3469),
 (11, 2143),
 (12, 2663),
 (13, 3067),
 (14, 2903),
 (15, 1471),
 (16, 3251),
 (17, 1583),
 (18, 1747),
 (19, 1259),
 (20, 1993),
 (21, 3391),
 (22, 3881),
 (23, 1931),
 (24, 1907),
 (25, 3329),
 (26, 1009),
 (27, 2011),
 (28, 3467),
 (29, 3533),
 (30, 3203),
 (31, 1307),
 (32, 1087),
 (33, 1879),
 (34, 1733),
 (35, 2357),
 (36, 3929),
 (37, 2621),
 (38, 1787),
 (39, 1549),
 (40, 1933),
 (41, 3557),
 (42, 3631),
 (43, 3877),
 (44, 2909),
 (45, 1873),
 (46, 3539),
 (47, 1657),
 (48, 2753),
 (49, 1409),
 (50, 2243),
 (51, 3541),
 (52, 3229),
 (53, 1019),
 (54, 1663),
 (55, 1567),
 (56, 3833),
 (57, 2551),
 (58, 1637),
 (59, 2281),
 (60, 3559),
 (61, 3511),
 (62, 2063),
 (63, 3187),
 (64, 3407),
 (65, 1597),
 (66, 2087),
 (67, 2749),
 (68, 1697),
 (69, 2711),
 (70, 2521),
 (71, 3163),
 (72, 3343),
 (73, 1303),
 (74, 3491),
 (75, 3767),
 (76, 1823),
 (77, 3863),
 (78, 18

In [322]:
# Generate the new canvas withour the boundaries
complete_canvas = np.zeros((8*n, 8*n), dtype=int)
for col in range(n):
    for row in range(n):
        j = col * n + row
        n_tile = canvas[j][0]
        print((j,n_tile))
        n_transform = canvas[j][1]
        i_tile = [x[0] for x in all_tiles if x[1] == n_tile][0]
        x = h.get_image(i_tile, data_all)
        m = h.image_to_matrix(x)
        m_all = h.transform(m)
        m_complete = m_all.iloc[n_transform - 1]
        m_complete_wo_boundaries = m_complete[1:-1, 1:-1]
        complete_canvas[(row*8):((row+1)*8), (col*8):((col+1)*8)] = m_complete_wo_boundaries

(0, 2243)
(1, 2707)
(2, 1481)
(3, 3559)
(4, 1489)
(5, 1583)
(6, 1307)
(7, 2549)
(8, 2677)
(9, 1289)
(10, 3511)
(11, 1213)
(12, 2663)
(13, 1549)
(14, 3821)
(15, 2161)
(16, 2003)
(17, 1471)
(18, 3769)
(19, 2207)
(20, 1933)
(21, 3329)
(22, 3187)
(23, 3259)
(24, 2087)
(25, 1409)
(26, 1399)
(27, 3833)
(28, 1873)
(29, 1601)
(30, 1801)
(31, 1019)
(32, 3853)
(33, 2237)
(34, 2099)
(35, 1531)
(36, 3067)
(37, 3343)
(38, 2477)
(39, 2609)
(40, 1831)
(41, 1063)
(42, 1097)
(43, 3631)
(44, 2551)
(45, 2591)
(46, 1787)
(47, 1867)
(48, 3881)
(49, 3491)
(50, 2753)
(51, 3557)
(52, 1657)
(53, 3923)
(54, 3643)
(55, 1597)
(56, 1259)
(57, 1931)
(58, 1979)
(59, 2441)
(60, 2333)
(61, 3467)
(62, 2063)
(63, 2843)
(64, 1447)
(65, 1879)
(66, 3209)
(67, 3767)
(68, 3541)
(69, 2357)
(70, 2711)
(71, 1621)
(72, 1607)
(73, 2411)
(74, 1009)
(75, 3037)
(76, 1733)
(77, 3313)
(78, 1181)
(79, 3251)
(80, 2749)
(81, 3929)
(82, 3023)
(83, 1667)
(84, 2111)
(85, 2909)
(86, 1187)
(87, 3539)
(88, 2957)
(89, 2287)
(90, 3863)
(91, 1993

In [323]:
# generate all shapes of the complete canvas
all_complete_canvas = h.transform(complete_canvas)

### Pattern elements

In [140]:
pattern = ['                  # ', '#    ##    ##    ###', ' #  #  #  #  #  #   ']

In [144]:
len(pattern[0])

20

In [145]:
# identify the elements that are '#'
pattern_position = []
for j, s_pattern in enumerate(pattern):
    for i, value in enumerate(s_pattern):
        if value == '#':
            pattern_position.append([j, i])


In [146]:
pattern_position

[[0, 18],
 [1, 0],
 [1, 5],
 [1, 6],
 [1, 11],
 [1, 12],
 [1, 17],
 [1, 18],
 [1, 19],
 [2, 1],
 [2, 4],
 [2, 7],
 [2, 10],
 [2, 13],
 [2, 16]]

In [147]:
len(pattern_position)

15

### Function to identify pattern 

The pattern is a matrix of shape (3,20)

In [227]:
def find_pattern(m, coordinates=pattern_position):
    rows, cols = zip(*coordinates)
    values = m[rows, cols]
    x = int(sum(values))
    return(x == 0) # all values = '#'

In [228]:
find_pattern(complete_canvas[:3, :20])

False

In [178]:
int(False)

0

### Run all matrix to find patters

In [282]:
all_complete_canvas.iloc[0].shape

(96, 96)

In [175]:
n

3

In [324]:
pattern_points = []
for k in range(8):
    m = all_complete_canvas.iloc[k]
    m_size = m.shape[0]
    total_pattern = 0
    m_size = m.shape[0]
    total_pattern = 0
    for i in range(m_size - 2):
        for j in range(m_size - 19):
            m_compare = m[i:(i+3), j:(j+20)] 
            if find_pattern(m_compare):
                total_pattern = total_pattern + int(find_pattern(m_compare))
                pattern_points = pattern_points + [(x[0] + i, x[1] + j) for x in pattern_position]
    print((total_pattern, m_size * m_size - int(sum(sum(m))) - total_pattern * 15))

(0, 2638)
(0, 2638)
(0, 2638)
(0, 2638)
(0, 2638)
(0, 2638)
(0, 2638)
(41, 2023)


In [315]:
len(set(pattern_points))

615

In [317]:
m_size * m_size

9216

In [318]:
int(sum(sum(m))) # (number of points)

6619

In [319]:
# number of '#'
9216 - 6619

2597

In [320]:
2597-615

1982

In [294]:
[[x[0] + 2, x[1]+3] for x in pattern_position]

[[2, 21],
 [3, 3],
 [3, 8],
 [3, 9],
 [3, 14],
 [3, 15],
 [3, 20],
 [3, 21],
 [3, 22],
 [4, 4],
 [4, 7],
 [4, 10],
 [4, 13],
 [4, 16],
 [4, 19]]

In [299]:
len(set([(1, 2), (3, 4), (1, 2)]))

2

array([[1, 1, 1, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 0, ..., 0, 1, 1],
       [1, 1, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 0, 1]], shape=(96, 96))

In [285]:
m.shape

(96, 96)

In [288]:
m_size * m_size

9216

In [286]:
total_pattern

41

In [289]:
int(sum(sum(m)))

6619

### Read matrix example

In [255]:
y = pd.read_csv("example_matrix_2.txt", header=None, names=['information'])
y = y['information'].iloc[:]
m_example = h.image_to_matrix(y)

In [257]:
m = m_example
m_size = m.shape[0]
total_pattern = 0
for i in range(m_size - 2):
    for j in range(m_size - 19):
#for i in range(2,3):
#    for j in range(2,3):
        m_compare = m[i:(i+3), j:(j+20)]
        total_pattern = total_pattern + int(find_pattern(m_compare))
print((total_pattern, m_size * m_size - int(sum(sum(m))) - total_pattern * 15))

(2, 273)
